In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64

In [4]:
df = pd.read_csv("data/depression.csv")
test_text = df["text"][4]
print(test_text)

sleep is my greatest and most comforting escape whenever i wake up these day the literal very first emotion i feel is just misery and reminding myself of all my problem i can t even have a single second to myself it s like waking up everyday is just welcoming yourself back to hell


In [5]:
def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")]))
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

In [6]:
def make_predictions(text):
    json_data = prepare_json(text)
    
    endpoint = "http://localhost:8080/v1/models/depression-detection-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    return "Depresi" if prediction > 0.6 else "Tidak Depresi"

In [7]:
make_predictions(test_text)

'Depresi'